# 03 — Signal EDA + windowing demo (ds003029)

Mục tiêu:
- Load 1 run có `*.eeg` thực sự (content present)
- QC nhanh tín hiệu (waveform + PSD)
- Demo windowing + feature table + label theo seizure intervals

Đầu vào:
- `eda_outputs/ds003029_run_summary.csv`
- `eda_outputs/ds003029_seizure_intervals_by_run.csv` (từ notebook 02)

Đầu ra:
- `eda_outputs/ds003029_window_features_demo.csv`
- `eda_outputs/ds003029_windowing_demo_info.csv`

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ws = Path.cwd().resolve()
src_dir = ws / 'src'
if not src_dir.exists() and (ws.parent / 'src').exists():
    ws = ws.parent
src_dir = (ws / 'src').resolve()
sys.path.insert(0, str(src_dir))
from ds003029_eda.paths import get_paths

try:
    import mne
except Exception as e:
    mne = None
    print('WARNING: mne not available -> cannot load BrainVision. Error:', e)

try:
    from scipy.signal import welch
except Exception as e:
    welch = None
    print('WARNING: scipy not available -> PSD will be skipped. Error:', e)

paths = get_paths()
run_summary = pd.read_csv(paths.outputs_dir / 'ds003029_run_summary.csv')
intervals = pd.read_csv(paths.outputs_dir / 'ds003029_seizure_intervals_by_run.csv')

print('run_summary:', run_summary.shape)
print('intervals:', intervals.shape)
plt.rcParams.update({'figure.dpi': 120})

In [ ]:
# Pick a run
assert mne is not None, 'Install mne to run signal-level EDA.'

rs = run_summary.copy()
interval_bases = set(intervals['base'].astype(str).tolist())
rs['has_interval'] = rs['base'].astype(str).isin(interval_bases)

cand = rs[rs['eeg_content_present'].astype(bool) & rs['has_interval']].copy()
if len(cand) == 0:
    cand = rs[rs['eeg_content_present'].astype(bool)].copy()

assert len(cand) > 0, 'No runs with eeg_content_present=1. Use git-annex get first.'
row = cand.sort_values(['n_channels'], ascending=False).head(1).iloc[0].to_dict()
base = str(row['base'])
vhdr = Path(base + '.vhdr')
assert vhdr.exists(), f'Missing {vhdr} (download .vhdr/.vmrk/.eeg first).'

In [ ]:
# Load & crop
raw = mne.io.read_raw_brainvision(vhdr, preload=False, verbose='ERROR')
sfreq = float(raw.info['sfreq'])
print(raw)
print('sfreq:', sfreq)

PRE_SEC = 120.0
POST_SEC = 120.0
MAX_CHANNELS = 16

run_int = intervals[intervals['base'].astype(str) == base].copy()
if len(run_int) > 0:
    t0 = max(0.0, float(run_int['onset_s'].min()) - PRE_SEC)
    t1 = float(run_int['offset_s'].max()) + POST_SEC
    t1 = min(float(raw.times[-1]), t1)
else:
    t0, t1 = 0.0, min(float(raw.times[-1]), 300.0)

seg = raw.copy().crop(tmin=t0, tmax=t1).load_data()
seg.pick(list(range(min(MAX_CHANNELS, len(seg.ch_names)))))
data = seg.get_data()
times = seg.times + t0
print('Segment data:', data.shape)

In [ ]:
# Waveform + PSD
plt.figure(figsize=(12, 6))
stack = np.nanstd(data) * 4.0 if np.nanstd(data) > 0 else 1.0
offsets = np.arange(data.shape[0]) * stack
for i in range(data.shape[0]):
    plt.plot(times, data[i] + offsets[i], linewidth=0.5)

if len(run_int) > 0:
    for _, r in run_int.iterrows():
        plt.axvspan(float(r['onset_s']), float(r['offset_s']), color='red', alpha=0.08)

plt.yticks(offsets, seg.ch_names)
plt.title('Waveforms (stacked) with ictal shading')
plt.xlabel('time (s)')
plt.tight_layout(); plt.show()

if welch is not None:
    plt.figure(figsize=(10, 4))
    nperseg = int(min(data.shape[1], sfreq * 2))
    for i in range(min(5, data.shape[0])):
        f, pxx = welch(data[i], fs=sfreq, nperseg=nperseg)
        plt.semilogy(f, pxx, alpha=0.7)
    plt.xlim(0, 250)
    plt.title('PSD (Welch) — first channels')
    plt.xlabel('Hz'); plt.ylabel('Power')
    plt.tight_layout(); plt.show()

In [ ]:
# Windowing + features + label
WINDOW_SEC = 2.0
STEP_SEC = 1.0

w = int(WINDOW_SEC * sfreq)
step = int(STEP_SEC * sfreq)
starts = list(range(0, data.shape[1] - w + 1, step))

def line_length(x: np.ndarray) -> float:
    return float(np.sum(np.abs(np.diff(x))))

interval_list = [(float(r['onset_s']), float(r['offset_s'])) for _, r in run_int.iterrows()]

rows = []
for s0 in starts:
    s1 = s0 + w
    win = data[:, s0:s1]
    t_mid = float(times[s0:s1].mean())

    rms = float(np.sqrt(np.nanmean(win ** 2)))
    ptp = float(np.nanmax(win) - np.nanmin(win))
    ll = float(np.nanmean([line_length(win[ch]) for ch in range(win.shape[0])]))

    y = 0
    for a, b in interval_list:
        if a <= t_mid <= b:
            y = 1
            break

    rows.append({'base': base, 't_mid_s': t_mid, 'rms': rms, 'ptp': ptp, 'line_length': ll, 'y': y})

feat = pd.DataFrame(rows)
display(feat.head())
print('windows:', len(feat), 'positive rate:', float(feat['y'].mean()) if len(feat) else float('nan'))

out_feat = paths.outputs_dir / 'ds003029_window_features_demo.csv'
out_info = paths.outputs_dir / 'ds003029_windowing_demo_info.csv'
feat.to_csv(out_feat, index=False)
pd.DataFrame([{
    'base': base,
    't0': t0,
    't1': t1,
    'sfreq': sfreq,
    'window_sec': WINDOW_SEC,
    'step_sec': STEP_SEC,
    'n_channels_used': int(data.shape[0]),
    'n_windows': int(len(feat)),
}]).to_csv(out_info, index=False)
print('Wrote:', out_feat.resolve())
print('Wrote:', out_info.resolve())